In [15]:
# Importações

import nltk

from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    cohen_kappa_score,
    roc_auc_score,
    confusion_matrix
)

import pandas as pd
import numpy as np

from nltk.corpus import stopwords

import re
import ast

# Cross validate

from sklearn.model_selection import cross_validate, cross_val_score

# Modelos

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

# Similaridade de Cossenos
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance

In [16]:
# Importação do Dataframe contendo títulos de notícias relacionadas a saúde.

df = pd.read_csv('dados/df_final.zip')

# Re-transforma as listas em listas
df['title'] = df['title'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['corpo_texto'] = df['corpo_texto'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

display(df)

,title,link,date,corpo_texto,noticia_falsa
0,"[verdade, casos, mortes, covid19, diminuíram, ...",http://www.e-farsas.com/e-verdade-que-os-casos...,Data não disponível,"[verdade, casos, mortes, covid19, diminuíram, ...",0.0
1,"[estudo, recomenda, uso, máscara, contra, covi...",https://projetocomprova.com.br/publicações/est...,2020/11/06,"[postagem, facebook, fonte, estudar, verdade, ...",0.0
2,"[covid, vai, parar, infectar, maioria, pessoas...",https://www.boatos.org/saude/covid-parar-quand...,26/11/2020,"[explicar, milésimo, vir, enfiar, cabeça, pess...",0.0
3,"[considerações, críticas, vacinas, contra, cov...",https://medicospelavidacovid19.com.br/geral/co...,Data não disponível,"[considerações, críticas, vacinas, contra, cov...",0.0
4,"[janssen, fará, reunião, dia, 16, pedir, uso, ...",https://noticias.uol.com.br/ultimas-noticias/r...,04/03/2021 22h21,"[janssen, subsidiário, johnson, johnson, pedir...",1.0
...,...,...,...,...,...
3330,"[cidade, lourenço, zerou, internação, covid, g...",https://www.e-farsas.com/a-cidade-de-sao-loure...,16/03/2021,"[publicação, espalhar, rede, social, whatsapp,...",0.0
3331,"[esquemas, vacinaisacesse, esquemas, vacinais,...",https://www.gov.br/saude/pt-br/assuntos/covid-...,Data não disponível,"[esquemas, vacinais, ministério, saúde, ir, co...",1.0
3332,"[saúde, autoriza, 45, leitos, uti, covid19, se...",https://www.gov.br/saude/pt-br/assuntos/notici...,14/04/2021 17h57,"[ministério, saudar, autorizar, n, quartafeira...",1.0
3333,"[fake, foto, mostre, coração, pessoa, após, us...",https://g1.globo.com/fato-ou-fake/coronavirus/...,01/09/2020,"[circular, rede, social, foto, coração, predom...",0.0


In [17]:
# Inicializa o vetorizador de TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Criação de vetores
X = df['title'].apply(lambda x: ' '.join(x))
y = df['noticia_falsa']

X = tfidf_vectorizer.fit_transform(X, y)

print("Dimensões de X:", X.shape)
print("Dimensões de y:", y.shape)

Dimensões de X: (3335, 6414)
Dimensões de y: (3335,)


In [18]:
# Define os modelos

# Árvore de Decisão
modelo_ad = DecisionTreeClassifier()

# Floresta Aleatória
modelo_rf = RandomForestClassifier()

# Modelo Regressão Logística
modelo_lr = LogisticRegression()

# SVC
# Parâmetros
C = 77
kernel = 'rbf'
gamma = 0.01

# Cria modelo
modelo_svm = svm.SVC(C=C, kernel=kernel, gamma=gamma)


## Definindo funções

In [19]:
stop_words = set(stopwords.words('portuguese'))

def trata_padrao(titulo, usar_links=False, link=None):
    '''
    Função para padronizar o tratamento dos títulos.

    Args:
    titulo (str) = título da notícia
    usar_links (bool) = usar ou não o link
    link (str) = link da notícia
    '''

    # Pré-processamento do titulo

    titulo = str(titulo).lower()
    titulo = nltk.word_tokenize(titulo)
    titulo = [word for word in titulo if word not in stop_words]
    titulo = [' '.join(titulo)]
    temp_tit = titulo

    # se usar link, pre processar link
    if usar_links:
        link = [split_and_clean(link)]
        temp_link = [' '.join(i) for i in link]
        print(temp_tit,temp_link)
        temp_tit_link = [' '.join(list(a)) for a in zip(temp_tit, temp_link)]
        processed = temp_tit_link
    else:
        processed = titulo

    return processed

In [20]:
# Define modelos para teste
modelos = [modelo_ad, modelo_rf, modelo_lr, modelo_svm]

# Treina modelos com todos os dados
for modelo in modelos:
    modelo.fit(X, y)

# Função para predizer novos títulos
def predict_title(titulo):
    preprocessed_title = trata_padrao(titulo)

    tfidf_vector = tfidf_vectorizer.transform(preprocessed_title)
    preds = dict()

    for modelo in modelos:
        preds[modelo] = modelo.predict(tfidf_vector)

    return preds

In [21]:
def comparar(df):
    '''
    Função para comparar predições de diferentes modelos com o target

    Args:
        df (pandas.df): dataframe com coluna '1' sendo os títulos/corpus, e '2' com o target/label
        
    Returns:
        list: Uma lista contendo 4 listas (uma para cada modelo: Decision Tree, Random Forest, Logistic Regression, SVC)
    '''
    DT, RF, LR, SVC = [], [], [], []
    for index, row in df.iterrows():
        text = row[0]
        target = row[1]
        pred = predict_title(text)
        pred['Target'] = [target]
        
        # Guardar as previsões em suas respectivas listas
        keys = [i for i in pred.keys()]
        DT.append(pred[keys[0]][0])
        RF.append(pred[keys[1]][0])
        LR.append(pred[keys[2]][0])
        SVC.append(pred[keys[3]][0])
        
        print(f'Notícia: {text}\n')
        for i in pred:
            print("\t{}\t{}".format(i, pred[i]))
        print('-----------------------------------------------------')
        print()
        
    return [DT, RF, LR, SVC]

### Similaridade de Cosseno

In [22]:
%matplotlib inline

In [23]:
# Função de similaridade de cosseno entre dois textos
def cosine_distance_countvectorizer_method(s1, s2):
    allsentences = [s1, s2]
    vectorizer = CountVectorizer()
    all_sentences_to_vector = vectorizer.fit_transform(allsentences)
    text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()

    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    return round((1 - cosine) * 100, 2)  # Similaridade em %

In [24]:
# Função principal que faz tudo: calcula similaridade, chama predict_title e gera gráfico
def process_title_and_predict(combined_df, palavra_especifica):
    similaridades = []

    # Calculando a similaridade da palavra específica com todas as outras
    for i, title in enumerate(combined_df['title']):
        if title != palavra_especifica:  # Não compara consigo mesma
            similaridade = cosine_distance_countvectorizer_method(palavra_especifica, title)
            noticia_falsa_status = combined_df.iloc[i]['noticia_falsa']  # Valor de 'noticia_falsa' correspondente
            similaridades.append((title, similaridade, noticia_falsa_status))

    # Ordenando as 10 palavras mais semelhantes
    similaridades_ordenadas = sorted(similaridades, key=lambda x: x[1], reverse=True)[:10]

    # Exibindo as 10 palavras mais semelhantes e o status de 'noticia_falsa'
    print(f"\nAs 10 títulos mais semelhantes a '{palavra_especifica}':")
    for title, sim, noticia_falsa in similaridades_ordenadas:
        print(f"{title} - Similaridade: {sim}% - Notícia Falsa: {noticia_falsa}")

    # Gráfico de barras para similaridades
    titles = [f'{title} - {noticia_falsa}' for title, sim, noticia_falsa in similaridades_ordenadas]
    sims = [sim for title, sim, noticia_falsa in similaridades_ordenadas]

    # Limpando o gráfico anterior
    plt.clf()

    # Plotando o gráfico
    plt.barh(titles, sims, color='skyblue')
    plt.xlabel('Similaridade (%)')
    plt.ylabel('Títulos')
    plt.title(f'Similaridade com: {palavra_especifica}')
    plt.gca().invert_yaxis()  # Inverte a ordem dos títulos no eixo Y para ficar mais intuitivo
    
    # Exibir o gráfico no notebook ou ambiente gráfico
    plt.show()

    # Chamada da função de predição (predict_title)
    print(f"\nPredição para o título: '{palavra_especifica}'")
    resultado_predicao = predict_title(palavra_especifica)
    print("Resultado da Predição:", resultado_predicao)

In [25]:
titulo_verdadeiro = input("Digite o título de uma notícia verdadeira para encontrar similaridades e prever: ")

process_title_and_predict(df, titulo_verdadeiro)

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
titulo_falso = input("Digite o título de uma notícia falsa para encontrar similaridades e prever: ")

process_title_and_predict(df, titulo_falso)